# UUID and Lineage Tracking in ASSYST

This notebook demonstrates how ASSYST tracks the derivation history of structures using UUIDs.

In [1]:
from assyst.crystals import pyxtal
from assyst.perturbations import Rattle, Stretch, Series
from assyst.relax import Relax
from assyst.calculators import Morse
import ase

2026-01-22 22:52:57.400305: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-22 22:52:57.419046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Initial Structure Generation

When we generate a structure with `pyxtal`, it automatically receives a UUID and a seed.

In [2]:
atoms = pyxtal(225, species=['Cu'], num_ions=[4])
print(f"Current UUID: {atoms.info['uuid']}")
print(f"Seed UUID:    {atoms.info['seed']}")
print(f"Lineage:      {atoms.info.get('lineage', [])}")

Current UUID: 880bfe98-991d-4189-bf81-e05480ab4b6b
Seed UUID:    880bfe98-991d-4189-bf81-e05480ab4b6b
Lineage:      []


## 2. Applying Perturbations

Perturbations create new structures with new UUIDs, while tracking the parent in the lineage.

In [3]:
rattle = Rattle(sigma=0.05)
perturbed = rattle(atoms.copy())

print(f"Current UUID: {perturbed.info['uuid']}")
print(f"Seed UUID:    {perturbed.info['seed']}")
print(f"Lineage:      {perturbed.info['lineage']}")

Current UUID: 5f09f358-e8ec-4e96-88d7-0a0ab99e8bac
Seed UUID:    880bfe98-991d-4189-bf81-e05480ab4b6b
Lineage:      ['880bfe98-991d-4189-bf81-e05480ab4b6b']


## 3. Multiple Steps

Lineage accumulates as we apply more modifications.

In [4]:
stretch = Stretch(hydro=0.05, shear=0.05)
final = stretch(perturbed.copy())

print(f"Current UUID: {final.info['uuid']}")
print(f"Seed UUID:    {final.info['seed']}")
print(f"Lineage:      {final.info['lineage']}")

Current UUID: 0ae01b8f-c0f3-4108-9cec-2051d14e9d46
Seed UUID:    880bfe98-991d-4189-bf81-e05480ab4b6b
Lineage:      ['880bfe98-991d-4189-bf81-e05480ab4b6b', '5f09f358-e8ec-4e96-88d7-0a0ab99e8bac']


## 4. Relaxation

Relaxation also updates the lineage.

In [5]:
calc = Morse()
relax = Relax(max_steps=5)

final.calc = calc.get_calculator()
relaxed = relax.relax(final)

print(f"Current UUID: {relaxed.info['uuid']}")
print(f"Seed UUID:    {relaxed.info['seed']}")
print(f"Lineage:      {relaxed.info['lineage']}")

Current UUID: 6cfd7b6f-5c82-483c-859d-a5e09b147175
Seed UUID:    880bfe98-991d-4189-bf81-e05480ab4b6b
Lineage:      ['880bfe98-991d-4189-bf81-e05480ab4b6b', '5f09f358-e8ec-4e96-88d7-0a0ab99e8bac', '0ae01b8f-c0f3-4108-9cec-2051d14e9d46']
